In [2]:
from scipy.optimize import fsolve

# Given parameters
iota = 1.6
bar_m = 1
target_value = 0.38

# Define the equation
def equation(theta):
    return (1 + theta ** (-iota)) ** (-1 / iota) - target_value / bar_m

# Solve for theta
theta_solution = fsolve(equation, 0.5)  # Initial guess of theta

print("The value of theta is approximately:", theta_solution[0])


The value of theta is approximately: 0.4412418920473948


# 1.3

In [19]:
import numpy as np
from scipy.optimize import root

# Define the matching function
def M(u, iota, m_):
    return m_ * (u ** (1 / iota))

# Define the syseq function
def syseq(x, params):
    p, beta, iota, gamma, z, s, b, m_, c_ = params
    expect = np.array([np.sum(np.array([p, 1 - p]) * x[2:4]), np.sum(np.array([p, 1 - p]) * x[3:2:-1])])
    out = np.zeros(4)
    out[0:2] = beta * M(1 / x[0:2], iota, m_) * expect - c_
    out[2:4] = (1 - gamma) * (z - b) - gamma * x[0:2] * c_ + beta * (1 - s) * expect - x[2:4]
    return out

# Define the simulation function
def simu(n, syseq, p, beta, iota, gamma, z, s, b, m_, c_):
    sol = root(syseq, x0=[0.5, 0.5, 0.5, 0.5], args=([p, beta, iota, gamma, z, s, b, m_, c_]))
    theta = sol.x[0:2]

    # Simulate the model
    i = np.random.choice([0, 1], size=n, p=[p, 1 - p])
    theta = theta[i]

    # Compute u_t, v_t, theta_t
    u = np.zeros(n)
    u[0] = 0.05
    for t in range(1, n):
        rfind = M(theta[t - 1], iota, m_)
        u[t] = u[t - 1] + s * (1 - u[t - 1]) - rfind * u[t - 1]

    v = theta * u

    return {'solution': sol, 'simulation': {'u': u, 'v': v, 'theta': theta}}

# Set parameters
n = 100000
p = 0.985
beta = 0.96 ** (1 / 12)
iota = 1.6
gamma = 0.5
z = np.exp([-0.01, 0.01])
s = 0.02
b = 0.4
m_ = 1
c_ = 1.210649  # Value obtained from previous calculation

# Run simulation
result = simu(n, syseq, p, beta, iota, gamma, z, s, b, m_, c_)
print("Solution:", result['solution'])
print("Simulation:", result['simulation'])


Solution:  message: The solution converged.
 success: True
  status: 1
     fun: [ 1.554e-15 -6.661e-16  0.000e+00  0.000e+00]
       x: [ 4.590e-01  4.745e-01  7.464e-01  7.624e-01]
    nfev: 11
    fjac: [[-8.481e-01 -5.554e-04 -5.299e-01 -4.208e-12]
           [-2.164e-04 -8.465e-01  1.234e-03 -5.324e-01]
           [ 5.299e-01 -7.200e-03 -8.480e-01  9.267e-03]
           [ 5.937e-03  5.324e-01 -1.006e-02 -8.464e-01]]
       r: [ 1.142e+00  2.647e-03 -1.400e+00 -6.785e-02  1.137e+00
            1.601e-02 -1.324e+00  9.198e-01  1.299e-02  8.603e-01]
     qtf: [ 5.419e-10 -2.965e-10 -3.412e-10  1.828e-10]
Simulation: {'u': array([0.05      , 0.03826705, 0.03398052, ..., 0.03151202, 0.03151263,
       0.03151286]), 'v': array([0.02295011, 0.01756466, 0.01559713, ..., 0.01446409, 0.01446437,
       0.01446447]), 'theta': array([0.45900219, 0.45900219, 0.45900219, ..., 0.45900219, 0.45900219,
       0.45900219])}


In [36]:
T = 6 
results = []
for i in range(T):
    result = simu(n, syseq, p, beta, iota, gamma, z, s, b, m_, c_)
    results.append(result['simulation'])

In [39]:
import pandas as pd

In [40]:
# Calculate standard deviation for each variable
std_u = [np.std(item['u']) for item in results]
std_v = [np.std(item['v']) for item in results]
std_theta = [np.std(item['theta']) for item in results]

# Create DataFrame
df = pd.DataFrame({'$\log u_t$': std_u, '$\log v_t$': std_v, '$\log \\theta_t$': std_theta})

# Export to LaTeX
latex_table = df.to_latex(index=False)

# Print LaTeX table
print(latex_table)

\begin{tabular}{rrr}
\toprule
 \$\textbackslash log u\_t\$ &  \$\textbackslash log v\_t\$ &  \$\textbackslash log \textbackslash theta\_t\$ \\
\midrule
   0.000083 &    0.000071 &         0.001912 \\
   0.000083 &    0.000071 &         0.001912 \\
   0.000083 &    0.000071 &         0.001900 \\
   0.000082 &    0.000070 &         0.001871 \\
   0.000082 &    0.000071 &         0.001890 \\
   0.000082 &    0.000070 &         0.001879 \\
\bottomrule
\end{tabular}



/var/folders/9t/k_w3tl715wg5gz461j6z69pc0000gn/T/ipykernel_71420/3714563724.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False)


# 1.5

In [53]:
# Run simulation
result = simu(n, syseq, p, 0.8 ** (1 / 12), iota, gamma, z, s, b, m_, c_)
print("Solution:", result['solution'])
print("Simulation:", result['simulation'])


Solution:  message: The solution converged.
 success: True
  status: 1
     fun: [ 3.464e-14 -2.043e-14 -1.110e-16  1.110e-16]
       x: [ 4.413e-01  4.564e-01  7.394e-01  7.554e-01]
    nfev: 11
    fjac: [[-8.468e-01 -1.763e-03 -5.319e-01 -4.455e-11]
           [-2.976e-04 -8.438e-01  3.270e-03 -5.366e-01]
           [ 5.318e-01 -1.313e-02 -8.466e-01  1.520e-02]
           [ 9.771e-03  5.364e-01 -1.733e-02 -8.437e-01]]
       r: [ 1.138e+00  6.936e-03 -1.435e+00 -7.997e-02  1.128e+00
            2.731e-02 -1.328e+00  9.640e-01  9.855e-03  8.898e-01]
     qtf: [ 7.052e-09 -4.550e-09 -4.506e-09  2.812e-09]
Simulation: {'u': array([0.05      , 0.03901422, 0.0348365 , ..., 0.03227287, 0.03227287,
       0.03227287]), 'v': array([0.02206392, 0.01721613, 0.01537259, ..., 0.01424132, 0.01424132,
       0.01424132]), 'theta': array([0.44127834, 0.44127834, 0.44127834, ..., 0.44127834, 0.44127834,
       0.44127834])}


In [58]:
T = 6 
results = []
for i in range(T):
    result = simu(n, syseq, p, 0.8 ** (1 / 12), iota, gamma, z, s, b, m_, c_)
    results.append(result['simulation'])

In [59]:
# Calculate standard deviation for each variable
std_u = [np.std(item['u']) for item in results]
std_v = [np.std(item['v']) for item in results]
std_theta = [np.std(item['theta']) for item in results]

# Create DataFrame
df = pd.DataFrame({'$\log u_t$': std_u, '$\log v_t$': std_v, '$\log \\theta_t$': std_theta})

# Export to LaTeX
latex_table = df.to_latex(index=False)

# Print LaTeX table
print(latex_table)

\begin{tabular}{rrr}
\toprule
 \$\textbackslash log u\_t\$ &  \$\textbackslash log v\_t\$ &  \$\textbackslash log \textbackslash theta\_t\$ \\
\midrule
   0.000081 &    0.000069 &         0.001816 \\
   0.000081 &    0.000069 &         0.001821 \\
   0.000081 &    0.000070 &         0.001846 \\
   0.000081 &    0.000069 &         0.001839 \\
   0.000082 &    0.000070 &         0.001850 \\
   0.000082 &    0.000070 &         0.001854 \\
\bottomrule
\end{tabular}



/var/folders/9t/k_w3tl715wg5gz461j6z69pc0000gn/T/ipykernel_71420/3714563724.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False)


# 1.6

In [61]:
import numpy as np
from scipy.optimize import root

# Define the matching function
def M(u, iota, m_):
    return m_ * (u ** (1/iota))

# Define the function for solving equations
def syseq_modified(x, params):
    p, beta, iota, gamma, z_l, z_h, s, m_, c_ = params
    b_t = 0.95 * np.exp(x[0])  # Time-varying flow utility
    expect = np.array([np.sum(np.array([p, 1 - p]) * x[2:4]), np.sum(np.array([p, 1 - p]) * x[3:2:-1])])
    out = np.zeros(4)
    out[0:2] = beta * M(1 / x[0:2], iota, m_) * expect - c_
    out[2:4] = (1 - gamma) * (np.exp(x[0]) - b_t) - gamma * x[0:2] * c_ + beta * (1 - s) * expect - x[2:4]
    return out

# Function for simulation
def simu_modified(n, p, beta, iota, gamma, z_l, z_h, s, m_, c_):
    sol = root(syseq_modified, x0=[0.5, 0.5, 0.5, 0.5], args=([p, beta, iota, gamma, z_l, z_h, s, m_, c_]))
    theta = sol.x[0:2]

    # Simulate the model
    i = np.random.choice([0, 1], size=n, p=[p, 1 - p])
    theta = theta[i]

    # Compute u_t, v_t, theta_t
    u = np.zeros(n)
    u[0] = 0.05
    for t in range(1, n):
        rfind = M(theta[t - 1], iota, m_)
        u[t] = u[t - 1] + s * (1 - u[t - 1]) - rfind * u[t - 1]

    v = theta * u

    return {'solution': sol, 'simulation': {'u': u, 'v': v, 'theta': theta}}

# Parameters
n = 100000
p = 0.985
beta = 0.96 ** (1 / 12)
iota = 1.6
gamma = 0.5
z_l = -0.01  # Log productivity level
z_h = 0.01   # Log productivity level
s = 0.02
m_ = 1
c_ = 1.210649  # Value obtained from previous calculation

# Simulation
result_modified = simu_modified(n, p, beta, iota, gamma, z_l, z_h, s, m_, c_)
print("Solution:", result_modified['solution'])
print("Simulation:", result_modified['simulation'])


Solution:  message: The solution converged.
 success: True
  status: 1
     fun: [ 2.165e-12 -1.227e-12  5.551e-17  8.327e-17]
       x: [ 3.690e-02  3.690e-02  1.545e-01  1.545e-01]
    nfev: 54
    fjac: [[-9.996e-01  2.856e-04 -2.739e-02  1.226e-03]
           [-3.201e-04 -9.996e-01 -2.194e-05 -2.869e-02]
           [ 2.741e-02 -1.123e-03 -9.988e-01  3.958e-02]
           [-1.314e-04  2.866e-02 -3.961e-02 -9.988e-01]]
       r: [ 2.116e+01 -1.691e-02 -7.701e+00 -8.448e-02  2.110e+01
            1.470e-02 -7.838e+00  2.492e-01 -2.199e-02  2.475e-01]
     qtf: [-3.540e-10  1.977e-10  9.926e-12 -5.721e-12]
Simulation: {'u': array([0.05      , 0.0626415 , 0.07342255, ..., 0.13589727, 0.13589727,
       0.13589727]), 'v': array([0.00184495, 0.00231141, 0.00270922, ..., 0.00501448, 0.00501448,
       0.00501448]), 'theta': array([0.03689907, 0.03689907, 0.03689907, ..., 0.03689907, 0.03689907,
       0.03689907])}


In [62]:
T = 6 
results = []
for i in range(T):
    result_modified = simu_modified(n, p, beta, iota, gamma, z_l, z_h, s, m_, c_)
    results.append(result_modified['simulation'])

In [63]:
# Calculate standard deviation for each variable
std_u = [np.std(item['u']) for item in results]
std_v = [np.std(item['v']) for item in results]
std_theta = [np.std(item['theta']) for item in results]

# Create DataFrame
df = pd.DataFrame({'$\log u_t$': std_u, '$\log v_t$': std_v, '$\log \\theta_t$': std_theta})

# Export to LaTeX
latex_table = df.to_latex(index=False)

# Print LaTeX table
print(latex_table)

\begin{tabular}{rrr}
\toprule
 \$\textbackslash log u\_t\$ &  \$\textbackslash log v\_t\$ &  \$\textbackslash log \textbackslash theta\_t\$ \\
\midrule
    0.00052 &    0.000019 &     2.880290e-15 \\
    0.00052 &    0.000019 &     2.933371e-15 \\
    0.00052 &    0.000019 &     2.859522e-15 \\
    0.00052 &    0.000019 &     2.859524e-15 \\
    0.00052 &    0.000019 &     2.894364e-15 \\
    0.00052 &    0.000019 &     2.834794e-15 \\
\bottomrule
\end{tabular}



/var/folders/9t/k_w3tl715wg5gz461j6z69pc0000gn/T/ipykernel_71420/3714563724.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False)
